# Covid19 Investigation

Set some key parameters:

In [27]:
mortality_rate = 8  # In %; see near the end for how I determined this
days_to_first_symptom = 5 # 5.4 days is the average value I have seen cited
days_from_first_symptom_to_death = 14  # again, the average

Import dependencies:

In [28]:
from datetime import date, datetime, timedelta
import os
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
import matplotlib.pyplot as plt 
import matplotlib.dates as dates
import ipywidgets as widgets
from scipy.optimize import curve_fit
from scipy.integrate import odeint


register_matplotlib_converters()
%matplotlib inline

Fetch the latest updated data from the Johns Hopkins repo.

In [29]:
df = None
last = datetime.now() - timedelta(hours=1)


def get_data():
    """
    Get the latest data. Just return the cached copy
    if less than one hour has elapsed.
    """
    global last, df
    now = datetime.now()
    if df and now - last < timedelta(hours=1):
        print('Using cached copy')
        return df

    if not os.path.exists('csse_covid_19_daily_reports'):
        os.mkdir('csse_covid_19_daily_reports')
        
    last = now    
    start = date(2020, 1, 22)
    end = date.today()

    df = pd.DataFrame()
    while start <= end:
        try:
            which = f'csse_covid_19_daily_reports/{start.month:02d}-{start.day:02d}-{start.year}.csv'
            if os.path.exists(which):
                df_day = pd.read_csv(which)
            else:
                df_day = pd.read_csv(f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/{which}')
                df_day.to_csv(which)
            df_day['Date'] = start
            if 'Country_Region' in df_day.columns:
                df_day = df_day.rename(columns={'Country_Region': 'Country/Region', 'Province_State': 'Province/State'})
            df = df.append(df_day)
        except Exception as e:
            print(f'{start}: {e}')
        start += timedelta(days=1)
    
    del df['Last Update']
    df = df.replace({'Country/Region': {'Mainland China': 'China'}})
    df.fillna({'Province/State': ''}, inplace=True)
    for col in ['Confirmed', 'Deaths', 'Recovered']:
        df.fillna({col: 0}, inplace=True)
    return df

In [30]:
df = get_data()
df.tail(5)

,Unnamed: 0,Province/State,Country/Region,Confirmed,Deaths,Recovered,Date,Latitude,Longitude,FIPS,...,Last_Update,Lat,Long_,Active,Combined_Key,Incident_Rate,People_Tested,People_Hospitalized,UID,ISO3
3218,NaN,,West Bank and Gaza,375.0,2.0,176.0,2020-05-07,NaN,NaN,NaN,...,2020-05-08 02:32:32,31.952200,35.233200,197.0,West Bank and Gaza,NaN,NaN,NaN,NaN,NaN
3219,NaN,,Western Sahara,6.0,0.0,5.0,2020-05-07,NaN,NaN,NaN,...,2020-05-08 02:32:32,24.215500,-12.885800,1.0,Western Sahara,NaN,NaN,NaN,NaN,NaN
3220,NaN,,Yemen,25.0,5.0,1.0,2020-05-07,NaN,NaN,NaN,...,2020-05-08 02:32:32,15.552727,48.516388,19.0,Yemen,NaN,NaN,NaN,NaN,NaN
3221,NaN,,Zambia,153.0,4.0,103.0,2020-05-07,NaN,NaN,NaN,...,2020-05-08 02:32:32,-13.133897,27.849332,46.0,Zambia,NaN,NaN,NaN,NaN,NaN
3222,NaN,,Zimbabwe,34.0,4.0,5.0,2020-05-07,NaN,NaN,NaN,...,2020-05-08 02:32:32,-19.015438,29.154857,25.0,Zimbabwe,NaN,NaN,NaN,NaN,NaN


Because the JH data is split by sub-regions (Province/State) we need to be able to aggregate it for countries. We also want to be able to handle it as either "daily new" or "cumulative". The helper function below can do this for us.

In [31]:
def aggregate_for_locations(df, locations, sub_locs=None, how=None, fields=None):
    if how is None:
        how = 'cumulative'
    if fields is None:
        fields = ['Confirmed', 'Deaths', 'Recovered']
    cols = ['Date']
    cols.extend(fields)
    
    if isinstance(locations, str):
        locations = [locations]
        
    if sub_locs:
        in_loc = df[(df['Country/Region'].isin(locations)) & (df['Province/State'].isin(sub_locs))]
    else:
        in_loc = df[df['Country/Region'].isin(locations)]
        
    result = in_loc[cols].groupby('Date').sum()
    
    if how[:3] == 'new':
        for f in fields:
            result[f] = result[f].diff()

    if how.find('-') > 0 and how[how.find('-')+1:] == '7day':  # moving average
        rolling = result.rolling(window=7)
        for f in fields:
            result[f] = rolling[f].mean()
    for f in fields:
        result.fillna({f: 0}, inplace=True)        
    return result


Helper function to plot a time series chart:

In [32]:
def plot_time_series(df, title):
    fig, ax = plt.subplots()
    ax.plot_date(df.index, df, 'v-')
    ax.xaxis.set_minor_locator(dates.WeekdayLocator(byweekday=(1), interval=1))
    ax.xaxis.set_minor_formatter(dates.DateFormatter('%d\n%a'))
    ax.xaxis.grid(True, which="minor")
    ax.yaxis.grid()
    ax.xaxis.set_major_locator(dates.MonthLocator())
    ax.xaxis.set_major_formatter(dates.DateFormatter('\n\n\n%b\n%Y'))
    ax.legend(df.columns, loc='upper left', shadow=True)
    ax.set_title(title)
    plt.tight_layout()
    return plt

Get the set of possible Country/Region values:

In [33]:
locations = sorted(list(df['Country/Region'].unique()))

We have one chart that lets us drill down into US states so we need the list of those. In some cases the data is broken into cities/counties, so we build up a dictionary of state abbreviations to the set of locations used for that state:

In [34]:
states = {
    'AL': ['Alabama'],
    'AK': ['Alaska'],
    'AZ': ['Arizona'],
    'AR': ['Arkansas'],
    'CA': ['California'],
    'CO': ['Colorado'],
    'CT': ['Connecticut'],
    'D.C.': ['District of Columbia'],
    'DE': ['Delaware'],
    'FL': ['Florida'],
    'GA': ['Georgia'],
    'HI': ['Hawaii'],
    'ID': ['Idaho'],
    'IL': ['Illinois'],
    'IN': ['Indiana'],
    'IA': ['Iowa'],
    'KS': ['Kansas'],
    'KY': ['Kentucky'],
    'LA': ['Louisiana'],
    'ME': ['Maine'],
    'MD': ['Maryland'],
    'MA': ['Massachusetts'],
    'MI': ['Michigan'],
    'MN': ['Minnesota'],
    'MS': ['Mississippi'],
    'MO': ['Missouri'],
    'MT': ['Montana'],
    'NE': ['Nebraska'],
    'NV': ['Nevada'],
    'NH': ['New Hampshire'],
    'NJ': ['New Jersey'],
    'NM': ['New Mexico'],
    'NY': ['New York'],
    'NC': ['North Carolina'],
    'ND': ['North Dakota'],
    'OH': ['Ohio'],
    'OK': ['Oklahoma'],
    'OR': ['Oregon'],
    'PA': ['Pennsylvania'],
    'PR': ['Puerto Rico'],
    'RI': ['Rhode Island'],
    'SC': ['South Carolina'],
    'SD': ['South Dakota'],
    'TN': ['Tennessee'],
    'TX': ['Texas'],
    'UT': ['Utah'],
    'VT': ['Vermont'],
    'VA': ['Virginia'],
    'WA': ['Washington'],
    'WV': ['West Virginia'],
    'WI': ['Wisconsin'],
    'WY': ['Wyoming'],
    'U.S.': [] # other territories
}

# Extend the above with all the counties/cities that were broken out separately
for s in df[df['Country/Region'] == 'US']['Province/State'].unique():
    i = s.find(',')
    if i < 0:
        continue
    st = s[i+2:i+5].strip()
    if st not in states:
        st += '.'
        if st not in states:
            print(f'Failed with {s}/{st}')
            continue
    states[st].append(s)


Create a country multi-picker:

In [35]:
picker = widgets.SelectMultiple(
                        options=locations,
                        value=['US'],
                        description='Locations'
                )

Our first interactive chart is for one or more countries, cumulative cases:

In [36]:
widgets.interact(lambda Location: plot_time_series(aggregate_for_locations(df, Location, how='cumulative'), 
                                                   title=f'{" ".join(Location)}: Cumulative'),
                 Location=picker)

interactive(children=(SelectMultiple(description='Locations', index=(219,), options=(' Azerbaijan', 'Afghanist…

<function __main__.<lambda>(Location)>

As above, but new cases rather than cumulative:

In [37]:
widgets.interact(lambda Location: plot_time_series(aggregate_for_locations(df, Location, how='new'), 
                                                   title=f'{" ".join(Location)}: New'),
                 Location=picker)

interactive(children=(SelectMultiple(description='Locations', index=(219,), options=(' Azerbaijan', 'Afghanist…

<function __main__.<lambda>(Location)>

In [38]:
widgets.interact(lambda Location: plot_time_series(aggregate_for_locations(df, Location, how='new-7day'), 
                                                   title=f'{" ".join(Location)}: New (7-day MA)'),
                 Location=picker)

interactive(children=(SelectMultiple(description='Locations', index=(219,), options=(' Azerbaijan', 'Afghanist…

<function __main__.<lambda>(Location)>

There are so many infections its hard to see the deaths, so pull those out separately:

In [39]:
widgets.interact(lambda Location: plot_time_series(aggregate_for_locations(df, Location, fields=['Deaths'], how='new'), 
                                                   title=f'{" ".join(Location)}: New'),
                 Location=picker)

interactive(children=(SelectMultiple(description='Locations', index=(219,), options=(' Azerbaijan', 'Afghanist…

<function __main__.<lambda>(Location)>

A chart for cumulative cases in different US states:

In [40]:
picker2 = widgets.Select(
                        options=states.keys(),
                        value='WA',
                        description='States'
                )
widgets.interact(lambda Location: plot_time_series(aggregate_for_locations(df, 'US', sub_locs=states[Location], how='cumulative'), 
                                                   title=f'{Location}: Cumulative'),
                 Location=picker2)

interactive(children=(Select(description='States', index=48, options=('AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT'…

<function __main__.<lambda>(Location)>

In [41]:
widgets.interact(lambda Location: plot_time_series(aggregate_for_locations(df, 'US', sub_locs=states[Location], how='new'), 
                                                   title=f'{Location}: New'),
                 Location=picker2)

interactive(children=(Select(description='States', index=48, options=('AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT'…

<function __main__.<lambda>(Location)>

In [42]:
widgets.interact(lambda Location: plot_time_series(aggregate_for_locations(df, 'US', sub_locs=states[Location], how='new-7day'), 
                                                   title=f'{Location}: New (Rolling Weekly Average)'),
                 Location=picker2)

interactive(children=(Select(description='States', index=48, options=('AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT'…

<function __main__.<lambda>(Location)>

In [43]:
widgets.interact(lambda Location: plot_time_series(aggregate_for_locations(df, 'US', sub_locs=states[Location], fields=['Deaths'], how='new'), 
                                                   title=f'{Location}: New'),
                 Location=picker2)

interactive(children=(Select(description='States', index=48, options=('AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT'…

<function __main__.<lambda>(Location)>

Now we use the reported deaths to try to infer the actual number infected. We first different curves to the death counts and use that to extrapolate:

In [44]:
def exponential(x, a):
    return a ** x


def power(x, a):
    return x ** a


def sigmoid(x, L ,x0, k, b):
    y = L / (1 + np.exp(-k*(x-x0)))+b
    return (y)


def sigmoid_init(x, y):
    return [max(y), np.median(x), 1, min(y)] # initial guess


# It takes on average 5 days to show symptoms. Typically 14 days from that to death.
# So it seems reasonable to say that people infected on day x will die around day x + 19
# So if we extrapolate the death rate by 15 days, we can approximate how many are infected 
# now after adjusting for fatality rate.

def predict_from_death_rate(region=None, 
                            fatality_rate=mortality_rate, 
                            death_time=days_to_first_symptom+days_from_first_symptom_to_death, 
                            fn=exponential, 
                            init=None):
    if region is None:
        region = 'US'
        
    deaths = aggregate_for_locations(df, region, how='cumulative')['Deaths']
    deaths = list(deaths[deaths > 0])
    
    x = range(1, len(deaths)+1)
    y = deaths
    
    if not y:
        print(f"Insufficient data for {region}")
        return
    
    if init:
        p0 = init(x, y)
        a, pcov = curve_fit(fn, x, y, p0, method='trf')
    else:
        a, pcov = curve_fit(fn, x, y)

    #a, pcov = curve_fit(fn, x[:-1], y[:-1])  # useful for seeing if latest data point is still aligned with curve

    plt.figure()
    plt.plot(x, y, 'ko', label=f"{region} Actual Deaths")
    plt.plot(x, fn(x, *a), 'r-', label="Fitted Curve")
    plt.legend()
    plt.show()
    
    forecast_deaths = fn(x[-1] + death_time, *a)
    predicted_infected = forecast_deaths * 100 / fatality_rate
    print(f'Predicted infected {int(predicted_infected)} leading to {int(forecast_deaths)} deaths in {death_time} days\nParams: {", ".join([str(aa) for aa in a])}')
    print(f'One year deaths: {fn(356, *a)}')

In [45]:
widgets.interact(lambda Location: predict_from_death_rate(Location),
                 Location=picker)

interactive(children=(SelectMultiple(description='Locations', index=(219,), options=(' Azerbaijan', 'Afghanist…

<function __main__.<lambda>(Location)>

In [46]:
widgets.interact(lambda Location: predict_from_death_rate(Location, fn=power),
                 Location=picker)

interactive(children=(SelectMultiple(description='Locations', index=(219,), options=(' Azerbaijan', 'Afghanist…

<function __main__.<lambda>(Location)>

In [47]:
#### Sigmoid version should be better for places that are flattening the curve or reaching saturation

widgets.interact(lambda Location: predict_from_death_rate(Location, fn=sigmoid, init=sigmoid_init),
                 Location=picker)

interactive(children=(SelectMultiple(description='Locations', index=(219,), options=(' Azerbaijan', 'Afghanist…

<function __main__.<lambda>(Location)>

In [48]:
# Sanity check to make sure we loaded todays data
df[df['Date'] == date.today]

,Unnamed: 0,Province/State,Country/Region,Confirmed,Deaths,Recovered,Date,Latitude,Longitude,FIPS,...,Last_Update,Lat,Long_,Active,Combined_Key,Incident_Rate,People_Tested,People_Hospitalized,UID,ISO3


In [49]:
# Calculate actual mortality rate


def calc_mortality_rate(loc):
    data = aggregate_for_locations(df, loc)
    result = []
    end = datetime.today() + timedelta(days=1)
    end = date(end.year, end.month, end.day)
    try:
        data.loc[end]
    except Exception:
        end -= timedelta(days=1)  # don't have today yet

    for days in range(1, days_from_first_symptom_to_death + 1):
        start = end - timedelta(days=days)
        try:
            # Use a 3 day count for better average
            deaths, initial = 0, 0
            for delta in range(3):
                deaths += data.loc[end - timedelta(days=delta)]['Deaths']
                initial += data.loc[start - timedelta(days=delta)]['Confirmed']
                
            rate = deaths / initial if initial else 0
            result.append(
                {
                    'days': days, # average number of days between confirmation and death; 
                                  # this is not really known and varies between countries
                                  # hence we output a table. Countres like Italy are overwhelmed
                                  # and testing mostly very sick patients so the number is smaller
                                  # than countries like Germany or SK that are doing a lot of
                                  # pre-emptive testing.
                    'start': start, # average date of confirmation for cases that died on date 'end' and the two days prior
                    'end': end,   # date of death (plus the two days prior)
                    'confirmed': initial, # count of the confirmed cases in the 3 day window ending at 'start'
                    'deaths': deaths, # count of the deceased in the 3 day window ending at 'end'
                    'rate': rate  # effective mortality rate for this case
                }
            )
        except Exception:
            pass

    return pd.DataFrame(result)

widgets.interact(lambda Location: calc_mortality_rate(Location), Location=picker)


interactive(children=(SelectMultiple(description='Locations', index=(219,), options=(' Azerbaijan', 'Afghanist…

<function __main__.<lambda>(Location)>

In [50]:
# Another way of estimating fatality rate is to take #deaths and #recovered 2 weeks later
# (assuming patients need to be clear for two weeks to be declared recovered) and sum those 
# for the denominator. This is likely an overestimate as the recovery count will exclude people
# who were never tested and recovered. Would likely worked best in places with good testing
# regimens.

def calc_mortality_rate2(loc):
    data = aggregate_for_locations(df, loc)
    result = []
    end = datetime.today() + timedelta(days=1)
    end = date(end.year, end.month, end.day)
    try:
        data.loc[end]
    except Exception:
        end -= timedelta(days=1)  # don't have today yet

    result = []
    for days in range(14):
        start = end - timedelta(days=days)
    
        died, recovered = 0, 0
        # Use a 3 day count for better average
        for delta in range(3):
            died += data.loc[start - timedelta(days=delta)]['Deaths']
            recovered += data.loc[end - timedelta(days=delta)]['Recovered']
        if died or recovered:
            rate = died/(died + recovered) if died or recovered else 0
            result.append({
                'days': days,
                'died': died,
                'recovered': recovered,
                'rate': rate
            })
    return pd.DataFrame(result)


widgets.interact(lambda Location: calc_mortality_rate2(Location), Location=picker)

interactive(children=(SelectMultiple(description='Locations', index=(219,), options=(' Azerbaijan', 'Afghanist…

<function __main__.<lambda>(Location)>

In [51]:
def model_SIR(N, R0=6.7, days=19):
    # N - population size
    # beta - contact rate rate
    # R0 is infection rate.
    # Low estimate for R0 is 2.2; high is 6.7
    # See https://sarahwestall.com/covid-19-has-much-higher-r0-value-than-originally-reported/
    # days is number of days a person is infectious
    # Estimate beta. Someone is infectious for gamma days and in that time infect R0 others
    # We assume that all of those are susceptible.
    # So beta = R0 / days
    
    gamma = 1 / days
    beta = R0 / days
    I_0, R_0 = 1, 0 # Initial number of infected and recovered individuals, I_0 and R_0.
    S_0 = N - I_0 - R_0 # Everyone else, S_0, is susceptible to infection initially.

    # A grid of time points (in days)
    t = np.linspace(0, 1000 / R0, 1000 / R0)

    # The SIR model differential equations.
    def deriv(y, t, N, beta, gamma):
        S, I, R = y
        dSdt = -beta * S * I / N
        dIdt = beta * S * I / N - gamma * I
        dRdt = gamma * I
        return dSdt, dIdt, dRdt

    # Initial conditions vector
    y_0 = S_0, I_0, R_0
    # Integrate the SIR equations over the time grid, t.
    ret = odeint(deriv, y_0, t, args=(N, beta, gamma))
    S, I, R = ret.T
    return S, I, R, t


def plot_SIR(S, I, R, t, N):
    # Plot the data on three separate curves for S(t), I(t) and R(t)
    fig = plt.figure(facecolor='w')
    ax = fig.add_subplot(111, axisbelow=True) #axis_bgcolor='#dddddd', axisbelow=True)
    if N > 100000000:
        lbl = 'billions'
        scale = 1000000000
    elif N > 10000000:
        lbl = '100 millions'
        scale = 100000000
    elif N > 1000000:
        lbl = '10 millions'
        scale = 10000000
    elif N > 100000:
        lbl = 'millions'
        scale = 1000000
    elif N > 10000:
        lbl = '100ks'
        scale = 100000
    elif N > 1000:
        lbl = '10ks'
        scale = 10000
    else:
        lbl = '1000s'
        scale = 1000

    ax.plot(t, S/scale, 'b', alpha=0.5, lw=2, label='Susceptible')
    ax.plot(t, I/scale, 'r', alpha=0.5, lw=2, label='Infected')
    ax.plot(t, R/scale, 'g', alpha=0.5, lw=2, label='Recovered with immunity')
    ax.set_xlabel('Time /days')
    ax.set_ylabel(f'Number ({lbl})')
    ax.set_ylim(0,1.2)
    ax.yaxis.set_tick_params(length=0)
    ax.xaxis.set_tick_params(length=0)
    ax.grid(b=True, which='major', c='w', lw=2, ls='-')
    legend = ax.legend()
    legend.get_frame().set_alpha(0.5)
    for spine in ('top', 'right', 'bottom', 'left'):
        ax.spines[spine].set_visible(False)
    plt.show()

In [52]:
N = 1000000
S,I,R, t = model_SIR(N)
plot_SIR(S, I, R, t, N)

TypeError: object of type <class 'float'> cannot be safely interpreted as an integer.

In [ ]:
def get_best_SIR(loc):
    # In thousands:
    populations = {
        'China': 1400000,
        'India': 1360000,
        'US': 330000,
        'Indonesia': 267000,
        'Pakistan': 220000,
        'Brazil': 212000,
        'Nigeria': 206000,
        'Bangladesh': 168000,
        'Russia': 147000,
        'Mexico': 127000,
        'Japan': 126000,
        'Philippines': 108000,
        'Egypt': 100000
    }
    if loc in populations:
        scale = populations[loc]
    else:
        scale = 100000
    
    conf = aggregate_for_locations(df, loc, how='new')['Confirmed']
    last = conf[-1]
    
    candidates = []
    
    for N in range(scale*1000, scale*100000, scale*1000):
        R0 = 2.0
        while R0 < 7.0:
            S,I,R,t = model_SIR(N, R0)
            
            # Now find the closest value in I to the last value in conf
            # We'll treat this as
            # the point of correlation. Get that date.
            best = None
            for i in range(1, len(I)):
                if I[i-1] < last < I[i]:
                    best = i
                    break
                #elif I[i-1] > I[i]:  # on the descent; no-one is there yet
                #    break
            #print(f'N {N}, R0 {R0}, lenC {len(conf)}, best {best}, lenI {len(I)}')
            if not best or best < len(conf):  # Also not a good fit as we should include the start of the curve
                R0 += 0.2
                continue

            # Calculate MSE of the two series to this point
            mse = 0
            for i in range(len(conf)):
                #print(f'lenC {len(conf)}, i {i}, best {best}, lenI {len(I)}')
                delta = I[best-i] - conf.iloc[-i-1]
                mse += delta * delta
                
            candidates.append((mse, N, R0, best))
            R0 += 0.2

    candidates.sort(key=lambda tup: tup[0])
    N = candidates[0][1]
    R0 = candidates[0][2]
    best = candidates[0][3]
    S,I,R,t = model_SIR(N, R0)
    return S, I, R, t, N, R0, best, conf


In [ ]:
S, I, R, t, N, R0, best, conf = get_best_SIR('US')

In [ ]:
offset = best - len(conf)
offset = 0
plot_SIR(S[offset:], I[offset:], R[offset:], t, N)